# Build the Neural Network
Neural networks compise of layers/modules that perform operations on data.
The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to build your own neural network.
Every module in PyTorch subclasses the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules (layers).
This nested structure allows for building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.

ニューラルネットワークはデータに対して実行するlayerとmoduleから構成されている．
torch.nn名前空間は，ニューラルネットワークを組み立てるのに必要なブロックが全てビルドされている．
PyTorchのすべてのモジュールはnn.Moduleをサブクラス化する．
ニューラルネットワークはそれ自身が他のモジュール(レイヤー)を含んだモジュールになっている．
このネストされた構造によって，複雑なアーキテクチャを組み立てて管理することが出来る．

以下のセクションでは，FashionMNISTデータセットの画像を分類するために，ニューラルネットワークを組み立てる．


In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
print(torch.__version__)

1.8.1


## Get Device for Training
We want to be able to train our model on a hardware accelerator lile the GPU, if it is available.
Let's check to see if [torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) is available, else we continue to use the CPU.

もし可能なら，モデル訓練にはGPUのようなハードウェアアクセラレーションを使いたい．
torch.cudaが使えるかチェックし，使えないなら引き続きCPUを使う．

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device.'.format(device))

Using cpu device.


## Define the Class
We define our neural network by subclassing `nn.Module`, and initialize the neural network layers in `__init__`.
Every `nn.Module` subclass implements the operations on input data in the `forward` method.

`nn.Module`をサブクラス化して，ニューラルネットワークを定義し，`__init__`内でニューラルネットワークのレイヤーを初期化する．
全ての`nn.Module`サブクラスは`forward`メソッド内で，入力データに対する操作を実装する．

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_rel_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_rel_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`, and print its structure.

`NeuralNetwork`のインスタンスを作り，`device`にモデルを移し，構造を表示する

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_rel_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


To use the model, we pass it the input data.
This executes the model's `forward`, along with some [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call `mdoel.forward()` directory!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each calss.
We get the prediction probabilities by passing it through an instance of the `nn.Softmax` module.

モデルを使うために，入力データを渡す．
これにより，いくつかのバックグラウンド操作とともに，モデルの`forward`が実行される．
直接`model.forward()`をしてはいけない．

入力してモデルを呼び出すと，それぞれのクラスに対した生の予測値をもった10次元tensorが返される．
`nn.Softmax`モジュールのインスタンスを通して，予測確率を計算する．

In [6]:
X = torch.rand(1, 28, 28, device=device)
# print(f'X: {X}')
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f'Predicted class: {y_pred}')

Predicted class: tensor([9])


## Model Layers
Let's break down the layers in the FashionMNIST model.
To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see waht happends to it as we pass it through the network.

FashionMNISTモデルのレイヤーを分解してみよう．
そのために，28x28サイズの3つの画像を持ったミニバッチサンプルを作り，ネットワークに渡した時に何が起こるかを見る．

In [7]:
input_image = torch.rand(3, 28, 28)
print(input_image)
print(input_image.size())

tensor([[[0.8128, 0.3292, 0.2968,  ..., 0.8809, 0.1153, 0.7465],
         [0.0501, 0.6921, 0.8031,  ..., 0.5209, 0.0278, 0.4134],
         [0.2398, 0.5149, 0.4355,  ..., 0.7992, 0.4206, 0.2928],
         ...,
         [0.8844, 0.0121, 0.2719,  ..., 0.5298, 0.8784, 0.7293],
         [0.2733, 0.8275, 0.6309,  ..., 0.9957, 0.6732, 0.2083],
         [0.4098, 0.7956, 0.5423,  ..., 0.3188, 0.1732, 0.5116]],

        [[0.4582, 0.1576, 0.0280,  ..., 0.6725, 0.4204, 0.0593],
         [0.4355, 0.5572, 0.5439,  ..., 0.1069, 0.3926, 0.6804],
         [0.5919, 0.2988, 0.7558,  ..., 0.4591, 0.6902, 0.4303],
         ...,
         [0.8699, 0.2252, 0.4551,  ..., 0.6674, 0.8577, 0.9826],
         [0.0012, 0.3672, 0.8195,  ..., 0.0352, 0.1489, 0.7793],
         [0.9581, 0.1901, 0.6193,  ..., 0.3620, 0.2060, 0.4264]],

        [[0.7948, 0.3452, 0.5529,  ..., 0.6459, 0.9062, 0.9466],
         [0.2956, 0.3752, 0.6322,  ..., 0.8081, 0.8761, 0.5511],
         [0.5486, 0.7784, 0.5887,  ..., 0.5057, 0.9832, 0.

### nn.Flatten
We initialize the [nn.Flaten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) layer to convert wach 2D 28x28 image into a contiguous array of 784 pixec values (the minibatch dimension (at dim=0) is maintained).

nn.Flatenレイヤーを初期化して，2Dの28x28画像を784ピクセル値の連続した配列に変換する(ミニバッチの次元(=0)は維持される)．

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image)
print(flat_image.size())

tensor([[0.8128, 0.3292, 0.2968,  ..., 0.3188, 0.1732, 0.5116],
        [0.4582, 0.1576, 0.0280,  ..., 0.3620, 0.2060, 0.4264],
        [0.7948, 0.3452, 0.5529,  ..., 0.3089, 0.2237, 0.8330]])
torch.Size([3, 784])


### nn.Linear
The [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) is a module that applies a linear transformation on the input using its stored weights and biases.

linear layerはモジュールで，保存された重みとバイアスを用いて入力データを線形変換する．

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1)
print(hidden1.size())

tensor([[ 0.1770, -1.1567,  0.0379, -0.3887,  0.0970,  0.3951,  0.1034, -0.7620,
          0.2434,  0.2521, -0.2528,  0.2172,  0.5640,  0.2983,  0.2867, -0.5163,
         -0.1014,  0.2721,  0.4994, -0.3706],
        [-0.1621, -1.0222,  0.1181, -0.5159, -0.2740,  0.1259,  0.2653, -0.6011,
          0.1555,  0.0501, -0.2117,  0.0173,  0.8495,  0.1052,  0.3524,  0.0571,
         -0.2359,  0.3415,  0.3847, -0.2149],
        [-0.1070, -0.6417,  0.2445, -0.4113, -0.1074,  0.2096,  0.2330, -0.6409,
         -0.0306,  0.1443, -0.0565,  0.1530,  0.5799,  0.0259,  0.6064, -0.6184,
         -0.1025,  0.1217,  0.2010, -0.1655]], grad_fn=<AddmmBackward>)
torch.Size([3, 20])


### nn.ReLU
Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our linear layers, but there's other activations to introduce non-linearity in your model.

非線形活性化関数はモデルの入力と出力間の関係性を複雑にするものである．
非線形活性化関数を，線形変換後に非線形性を導入することで，ニューラルネットワークが様々な現象を学習するのを助ける．

このモデルでは，nn.ReLUを線形層の間で用いるが，他の非線形性を導入する活性化関数も用いることが出来る．

In [10]:
print(f'Before ReLu: {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)
print(f'After ReLU: {hidden1}')

Before ReLu: tensor([[ 0.1770, -1.1567,  0.0379, -0.3887,  0.0970,  0.3951,  0.1034, -0.7620,
          0.2434,  0.2521, -0.2528,  0.2172,  0.5640,  0.2983,  0.2867, -0.5163,
         -0.1014,  0.2721,  0.4994, -0.3706],
        [-0.1621, -1.0222,  0.1181, -0.5159, -0.2740,  0.1259,  0.2653, -0.6011,
          0.1555,  0.0501, -0.2117,  0.0173,  0.8495,  0.1052,  0.3524,  0.0571,
         -0.2359,  0.3415,  0.3847, -0.2149],
        [-0.1070, -0.6417,  0.2445, -0.4113, -0.1074,  0.2096,  0.2330, -0.6409,
         -0.0306,  0.1443, -0.0565,  0.1530,  0.5799,  0.0259,  0.6064, -0.6184,
         -0.1025,  0.1217,  0.2010, -0.1655]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.1770, 0.0000, 0.0379, 0.0000, 0.0970, 0.3951, 0.1034, 0.0000, 0.2434,
         0.2521, 0.0000, 0.2172, 0.5640, 0.2983, 0.2867, 0.0000, 0.0000, 0.2721,
         0.4994, 0.0000],
        [0.0000, 0.0000, 0.1181, 0.0000, 0.0000, 0.1259, 0.2653, 0.0000, 0.1555,
         0.0501, 0.0000, 0.0173, 0.8495, 0.1052, 0.352

### nn.Sequentioal
[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an orderd container of modules.
The data is passed through all the modules in the same order as defined.
You can use sequential containers to put togegher a quick network lile `seq_modelues`.

nn.Sequentialはモジュールの順序づけられたコンテナである．
データは定義された順番に全てのモジュールを通る．
シーケンシャルコンテナを使用して，`seq_modules`のようなクイックネットワークをまとめることができる

In [11]:
seq_modules = nn.Sequential(
    flatten, 
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
print(logits)

tensor([[ 2.9027e-01,  2.3255e-01, -2.9550e-04,  5.4221e-01,  7.0530e-02,
         -2.8926e-01, -1.2192e-01,  2.0138e-01, -3.1618e-01, -4.0473e-01],
        [ 2.7024e-01,  1.7551e-01,  7.9698e-02,  3.3475e-01,  1.2269e-02,
         -2.8137e-01, -1.8714e-01,  8.8177e-02, -2.9120e-01, -1.9627e-01],
        [ 3.2363e-01,  2.0389e-01,  1.7663e-02,  4.5542e-01,  8.2976e-02,
         -2.5510e-01, -1.4627e-01,  1.6944e-01, -3.2762e-01, -2.6919e-01]],
       grad_fn=<AddmmBackward>)


### nn.Softmax
The last linear layer of the neural network returns logits - raw values in \[-$\infty$, $\infty$\] - which are passed to the [nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module.
The logtis are scaled to vlaues \[0, 1\] representing the model's predicted probabilities for each class. 
`dim` parameter indicates the dimension along which the values must sum to 1.

ニューラルネットワークの最後の線形層はロジット\[-$\infty$, $\infty$\]を返し，これをnn.Softmaxモジュールに渡す．
ロジットはそれぞれのクラスを予測する確率を表現する\[0, 1\]にスケーリングされる．
`dim`パラメータは合計が1になる次元を指す．

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

# predicted class
pred_class = pred_probab.argmax(dim=1)
print(pred_class)

tensor([[0.1256, 0.1185, 0.0939, 0.1616, 0.1008, 0.0703, 0.0832, 0.1149, 0.0685,
         0.0627],
        [0.1280, 0.1164, 0.1058, 0.1365, 0.0989, 0.0737, 0.0810, 0.1067, 0.0730,
         0.0802],
        [0.1304, 0.1157, 0.0960, 0.1488, 0.1025, 0.0731, 0.0815, 0.1118, 0.0680,
         0.0721]], grad_fn=<SoftmaxBackward>)
tensor([3, 3, 3])


## Model Parameters
Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training.
Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model's `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

ニューラルネットワークの多くのレイヤーはパラメータ化されている．
つまり，訓練の間に最適化された重みとバイアスが関連づけられている．
`nn.Module`をサブクラス化することで，モデル内で定義された全てのフィールドを自動的に追跡し，モデルが持つ`parameters()`や`names_parameters()`メソッドを用いることで全てのパラメータにアクセスすることができる．

この例では，全てのパラメータを反復処理し，サイズと値を表示する．

In [13]:
print('Moel structure: ', model, '\n\n')

for name, param in model.named_parameters():
    print(f'Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n')

Moel structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_rel_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_rel_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0223,  0.0096, -0.0141,  ..., -0.0350, -0.0044, -0.0019],
        [ 0.0065,  0.0278, -0.0346,  ..., -0.0278, -0.0290, -0.0198]],
       grad_fn=<SliceBackward>) 

Layer: linear_rel_stack.0.bias | Size: torch.Size([512]) | Values: tensor([ 0.0142, -0.0274], grad_fn=<SliceBackward>) 

Layer: linear_rel_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0336, -0.0434, -0.0097,  ..., -0.0005,  0.0077, -0.0223],
        [-0.0033,  0.0042, -0.0266,  ..., -0.0021,  0.0338, -0.0058]],
       grad_fn=<SliceBackward>) 

Layer: linear_rel_stack.2.b

## Further Reading
- [torch.nn API](https://pytorch.org/docs/stable/nn.htmlj)